In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [57]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [49]:
import requests
from bs4 import BeautifulSoup
from langchain.agents import tool


@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


@tool
def add_function(a: float, b: float) -> float:
    """Adds two numbers together."""
    return a + b


@tool
def naver_news_crawl(news_url: str) -> str:
    """Crawls a 네비어(naver.com) news article and returns the body content."""
    # HTTP GET 요청 보내기
    response = requests.get(news_url)

    # 요청이 성공했는지 확인
    if response.status_code == 200:
        # BeautifulSoup을 사용하여 HTML 파싱
        soup = BeautifulSoup(response.text, "html.parser")

        # 원하는 정보 추출
        title = soup.find("h2", id="title_area").get_text()
        content = soup.find("div", id="contents").get_text()

        # 출력
        print(f"기사 제목: {title}")
        print(f"본문 내용:\n{content}")
    else:
        print(f"HTTP 요청 실패. 응답 코드: {response.status_code}")
    return content


tools = [get_word_length, add_function, naver_news_crawl]

In [50]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [51]:
from langchain_community.tools.convert_to_openai import format_tool_to_openai_function

llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

In [52]:
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

In [53]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [36]:
agent_executor.invoke({"input": "How many letters in the word educa"})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5There are 5 letters in the word "educa".

> Finished chain.


{'input': 'How many letters in the word educa',
 'output': 'There are 5 letters in the word "educa".'}

In [37]:
agent_executor.invoke({"input": "114.5 + 121.2"})



> Entering new AgentExecutor chain...

Invoking: `add_function` with `{'a': 114.5, 'b': 121.2}`


235.7The sum of 114.5 and 121.2 is 235.7.

> Finished chain.


{'input': '114.5 + 121.2', 'output': 'The sum of 114.5 and 121.2 is 235.7.'}

In [38]:
agent_executor.invoke({"input": "114.5 + 121.2 + 34.2 + 110.1"})



> Entering new AgentExecutor chain...

Invoking: `add_function` with `{'a': 114.5, 'b': 121.2}`


235.7
Invoking: `add_function` with `{'a': 235.7, 'b': 34.2}`


269.9
Invoking: `add_function` with `{'a': 269.9, 'b': 110.1}`


380.0The sum of 114.5, 121.2, 34.2, and 110.1 is 380.0.

> Finished chain.


{'input': '114.5 + 121.2 + 34.2 + 110.1',
 'output': 'The sum of 114.5, 121.2, 34.2, and 110.1 is 380.0.'}

In [56]:
agent_executor.invoke(
    {
        "input": "뉴스 기사를 요약해 줘: https://n.news.naver.com/mnews/article/277/0005366099?sid=101"
    }
)



> Entering new AgentExecutor chain...

Invoking: `naver_news_crawl` with `{'news_url': 'https://n.news.naver.com/mnews/article/277/0005366099?sid=101'}`


기사 제목: 뉴욕증시, 예상 웃돈 CPI에 장초반 하락세
본문 내용:



			미국 뉴욕증시의 3대 지수는 예상보다 강한 12월 소비자물가지수(CPI) 여파로 11일(현지시간) 장 초반 하락세를 나타내고 있다.이날 뉴욕증권거래소(NYSE)에서 오전 10시50분께 다우존스30산업평균지수는 전장 대비 0.45% 내린 3만7524선에서 거래 중이다. 대형주 중심의 S&P500지수는 0.49% 낮은 4759선, 기술주 중심의 나스닥 지수는 0.52% 하락한 1만4891선을 기록 중이다.현재 S&P500지수에서 에너지 관련주를 제외한 나머지 10개 업종이 모두 하락하고 있다. 애플은 이날도 1%에 가까운 낙폭을 기록 중이다. 애플은 올 들어 약세를 지속하면서 이날 한때 마이크로소프트에게 시가총액 1위 자리를 내주기도 했다. 테슬라는 3%이상 밀렸다. 코인베이스와 로빈후드는 투자자들이 전날 장 마감후 미국 증권거래위원회(SEC)가 발표한 비트코인 현물 ETF가 승인 여파를 평가하면서 각각 2~3%대 낙폭을 기록 중이다. 다음날 실적 발표를 앞둔 시티그룹은 아르헨티나 페소 하락, 구조조정 비용 등으로 인해 분기손실이 발생할 수 있다는 경고로 2%이상 내렸다. 리프트는 골드만삭스가 매수에서 중립으로 투자의견을 하향한 후 3%이상 하락했다. 구글 알파벳은 엔지니어링과 하드웨어 부문 직원을 수백명 감원한다는 소식에 소폭 올랐다.



[이미지출처=로이터연합뉴스]투자자들은 이날 개장전 공개된 12월 CPI를 소화하며 향후 연방준비제도(Fed)의 통화정책 향방을 가늠하고자하고 있다. 누적된 통화긴축 효과에 힘입어 디스인플레이션 추세를 보여왔던 CPI 상승률은 여전히 높은 주거비 등의 여파로 전년 대비 3

{'input': '뉴스 기사를 요약해 줘: https://n.news.naver.com/mnews/article/277/0005366099?sid=101',
 'output': '미국 뉴욕증시의 3대 지수는 예상보다 강한 12월 소비자물가지수(CPI) 여파로 11일(현지시간) 장 초반 하락세를 나타내고 있다. 이날 뉴욕증권거래소(NYSE)에서 오전 10시50분께 다우존스30산업평균지수는 전장 대비 0.45% 내린 3만7524선에서 거래 중이다. 대형주 중심의 S&P500지수는 0.49% 낮은 4759선, 기술주 중'}